In [7]:
from langchain.llms.gpt4all import GPT4All
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "You are a helpful assistant that defines words. Define this word: {word}.",
)

llm = GPT4All(
    model="./falcon.bin",
)

chain = prompt | llm

chain.invoke({
    "word": "tomato",
})

ValidationError: 1 validation error for GPT4All
__root__
  Failed to retrieve model (type=value_error)